In [ ]:
import weave
from weave import weave_internal
from weave.ecosystem.all import langchain
weave.use_frontend_devmode()

In [ ]:
weave.panels.Board(
    vars={
        "chat_model": langchain.chat_openai("gpt-3.5-turbo", 0.7),
        "artists" : ["René Magritte", "Josephine Wall", "Android Jones", "Alphonse Mucha", "Claude Monet"],
        "artist_id" : 0,
        "num_works": 3,
        "palette" : ["red", "orange", "yellow", "green", "blue", "violet"],
        "style_riff": 
            lambda chat_model, num_works, artists, palette, artist_id:
                weave.ops.range(0, num_works, 1).map(
                    lambda row:
                        chat_model.predict(weave_internal.const("You are an expert art historian. In a sophisticated, avant-garde art museum, you see pieces in ")
                            + palette[row] + " by " + artists[artist_id]
                            + ", which has a fantastic contrast and synergy with the work of")
      
        ),
        "riff_art":
            lambda style_riff, num_works:
                weave.ops.range(0, num_works, 1).map(
                    lambda row: style_riff[row]).stable_diffusion(),
        "art_desc":
            lambda riff_art, num_works:
                weave.ops.range(0, num_works, 1).map(
                    lambda row: riff_art[row]).img2prompt(),
        "desc_title":
            lambda art_desc, num_works, chat_model:
                weave.ops.range(0, num_works, 1).map(
                    lambda row: chat_model.predict(weave_internal.const("Art description: ")
                                + art_desc[row] + ". Perfect title:")),
        "R" : 168,
        "G" : 156,
        "B" : 234,
        "color" : lambda chat_model, R, G, B:
                        chat_model.predict(weave_internal.const("RGB value: (")
                        + R.toString() + "," + G.toString() + "," + B.toString()
                        + "). Precise color name:"),
        "remixed_art":
            lambda desc_title, num_works, color:
                weave.ops.range(0, num_works, 1).map(
                    lambda row: weave_internal.const("a beautiful painting titled ")
                                 + desc_title[row] + " with more " + color).stable_diffusion()
                                                                    
    },
    panels=[
        weave.panels.BoardPanel(
            lambda style_riff: weave.panels.Table(style_riff, columns=[lambda col: col]),
            layout=weave.panels.BoardPanelLayout(x=0, y=0, w=4, h=16)
        ),
        weave.panels.BoardPanel(
            lambda riff_art: weave.panels.Table(riff_art, columns=[lambda col: col]),
            layout=weave.panels.BoardPanelLayout(x=4, y=0, w=6, h=16)
        ),
        weave.panels.BoardPanel(
            lambda art_desc: weave.panels.Table(art_desc, columns=[lambda col: col]),
            layout=weave.panels.BoardPanelLayout(x=10, y=0, w=4, h=16)
        ),
        weave.panels.BoardPanel(
            lambda desc_title: weave.panels.Table(desc_title, columns=[lambda col: col]),
            layout=weave.panels.BoardPanelLayout(x=14, y=0, w=4, h=16)
        ),
        weave.panels.BoardPanel(
            lambda remixed_art: weave.panels.Table(remixed_art, columns=[lambda col: col]),
            layout=weave.panels.BoardPanelLayout(x=18, y=0, w=6, h=16)
        ),
        weave.panels.BoardPanel(
            lambda color: weave.panels.StringEditor(color),
            layout=weave.panels.BoardPanelLayout(x=7, y=16, w=10, h=4)
        ),
        
    ]
)